In [1]:
import pathlib
import pandas as pd
import shutil
import logging

from IDR_stream.IdrStream import IdrStream

In [2]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
# remove A1 wells because of irregular illumination
data_to_process = data_to_process[data_to_process["Well"] != "A1"]
data_to_process = data_to_process.reset_index(drop=True)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target
0,LT0001_02,A4,50,4,positive control,ENSG00000149503
1,LT0001_02,A15,50,15,negative control,negative control
2,LT0001_02,B2,50,26,negative control,negative control
3,LT0001_02,C1,50,49,positive control,ENSG00000149503
4,LT0001_02,C4,50,52,positive control,ENSG00000149503
...,...,...,...,...,...,...
6738,LT0603_06,O13,50,349,positive control,KIF11
6739,LT0603_06,O16,50,352,negative control,negative control
6740,LT0603_06,P3,50,363,negative control,negative control
6741,LT0603_06,P21,50,381,positive control,COPB


In [3]:
idr_id = "idr0013"
tmp_dir = pathlib.Path("tmp/")
final_data_dir = pathlib.Path("data/")
try:
    shutil.rmtree(tmp_dir)
    #shutil.rmtree(final_data_dir)
except:
    print("No files to remove")

stream = IdrStream(idr_id, tmp_dir, final_data_dir, log=True)

In [4]:
aspera_path = pathlib.Path("/home/roshankern/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
save_dir = pathlib.Path("data/")

stream.init_downloader(aspera_path, aspera_key_path, screens_path)

In [5]:
fiji_path = pathlib.Path("/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app")
stream.init_preprocessor(fiji_path)

[INFO] Overriding Leica ROI Reader; identifier: command:de.biovoxxel.utilities.RoiReader; jar: file:/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app/plugins/Biovoxxel_Plugins-2.5.6.jar


[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


In [6]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

>>> GPU activated? 1


In [7]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [8]:
stream.run_stream(data_to_process, batch_size=10, start_batch=0)

Completed: 75739K bytes transferred in 3 seconds
 (172824K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0001_02/00004_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00004/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 87703K bytes transferred in 4 seconds
 (171194K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0001_02/00015_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00015/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 74464K bytes transferred in 3 seconds
 (161465K bits/sec), in 1 file.
CellH5Reader initializi

2022-08-11 15:15:57.222157: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
/home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.3 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/roshankern/D

2022-08-11 15:15:58,341 - WARNING - From /home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form tmp/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  10 non-null     object
 1   Metadata_Well   10 non-null     object
 2   Metadata_Site   10 non-null     int64 
 3   Plate_Map_Name  10 non-null     object
 4   DNA             10 non-null     object
 5   Gene            10 non-null     object
 6   Gene_Replicate  10 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 688.0+ bytes
None
{'ENSG000001495

2022-08-11 15:16:00.018132: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-11 15:16:00.041442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 15:16:00.041722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.882GHz coreCount: 28 deviceMemorySize: 11.76GiB deviceMemoryBandwidth: 335.32GiB/s
2022-08-11 15:16:00.041735: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-08-11 15:16:00.043992: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-08-11 15:16:00.044022: I tensorflow/stream_executor/pl

2022-08-11 15:16:01,177 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream_prototype/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

...

2022-08-11 15:38:35,070 - WARNING - From /home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form tmp/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  10 non-null     object
 1   Metadata_Well   10 non-null     object
 2   Metadata_Site   10 non-null     int64 
 3   Plate_Map_Name  10 non-null     object
 4   DNA             10 non-null     object
 5   Gene            10 non-null     object
 6   Gene_Replicate  10 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 688.0+ bytes
None
{'ENSG000001495

2022-08-11 15:38:36.763426: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-11 15:38:36.785244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 15:38:36.785516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.882GHz coreCount: 28 deviceMemorySize: 11.76GiB deviceMemoryBandwidth: 335.32GiB/s
2022-08-11 15:38:36.785530: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-08-11 15:38:36.787776: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-08-11 15:38:36.787807: I tensorflow/stream_executor/pl

2022-08-11 15:38:37,910 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream_prototype/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

/home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-08-11 15:38:38.548948: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-08-11 15:38:39.127398: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8401
2022-08-11 15:38:39.830677: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-11 15:38:39.830703: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2022-08-11 15:38:39.831265: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or dire

LT0003_40/4_50-1 (180 cells) : 2.96 secs
LT0003_40/15_50-1 (159 cells) : 1.16 secs
LT0003_40/26_50-1 (187 cells) : 1.27 secs
LT0003_40/49_50-1 (248 cells) : 1.64 secs
LT0003_40/52_50-1 (147 cells) : 1.11 secs
LT0003_40/63_50-1 (123 cells) : 0.96 secs
LT0003_40/74_50-1 (107 cells) : 0.89 secs
LT0003_40/304_50-1 (58 cells) : 0.63 secs
LT0003_40/315_50-1 (156 cells) : 1.17 secs
LT0003_40/352_50-1 (210 cells) : 1.45 secs
Profiling: done
Completed: 69998K bytes transferred in 3 seconds
 (163064K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0004_06/00004_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0004_06--ex2005_11_18--sp2005_03_18-tt173--c3/experiment/
Site :/sample/0/plate/LT0004_06--ex2005_11_18--sp2005_03_18-tt173--c3/experiment/00004/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 87506K bytes transferred in 3 seconds
 (202949K bits/sec), in 1 file.
CellH5Reader initializing /home/ro